In [1]:
%%capture
!pip install ../input/keras-applications/Keras_Applications-1.0.8/ -f ./ --no-index
!pip install ../input/image-classifiers/image_classifiers-1.0.0/ -f ./ --no-index
!pip install ../input/efficientnet-1-0-0/efficientnet-1.0.0/ -f ./ --no-index
!pip install ../input/segmentation-models/segmentation_models-1.0.1/ -f ./ --no-index

In [2]:
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [3]:
%%capture
import os
import gc
import cv2
import pdb
import glob
import pytz
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
import segmentation_models as sm
import warnings

import rasterio
from rasterio.windows import Window

In [4]:
height, width = 1024, 1024
reduce = 2
THRESHOLD = 0.55
MIN_PRED = 2000
BORDER_ZEROS = 64
VOTERS = 0.50 # increase VOTERS? - Since Multiple glances at each slide in theory.
window = 2048
min_overlap = 512
test_all = True
files = ['2ec3f1bb9', '3589adb90', 'd488c759a', 'aa05346ff', '57512b7f1']
DATA = '../input/hubmap-kidney-segmentation/test/'
MODELS = [
    '../input/tpu-model/model-fold-0.h5',
    
    #'../input/linknet-folds/model-fold-0_dice.h5',
    '../input/linknet-folds/model-fold-1_th.h5',
    '../input/linknet-folds/model-fold-2_th.h5',
    '../input/linknet-folds/model-fold-3_th.h5',
    '../input/linknet-folds/model-fold-4_th.h5',
    
]
use_TTA = True

df_sample = pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')
STRATEGY = tf.distribute.get_strategy() 
batch_size = 1

In [5]:
P = {}
P['NUM_CLASSES'] = 2
class Model(tf.keras.Model):
    def __init__(self, model_type, backbone):
        super().__init__()
        self.model_type = model_type
        if self.model_type == 'unet':
            self.model = sm.Unet(backbone, activation = 'linear', classes = P['NUM_CLASSES'], encoder_weights=None)
        elif self.model_type == 'fpn':
            self.model = sm.FPN(backbone, activation = 'linear', classes = P['NUM_CLASSES'], encoder_weights=None)
        else:
            self.model = sm.Linknet(backbone, activation = 'linear', classes = P['NUM_CLASSES'], encoder_weights=None)
        self.layer = tf.keras.layers.Layer(dtype = tf.float32)
    def call(self, x, training = True):
        if use_TTA:
            flip0 = tf.reverse(x, axis = [1])
            flip1 = tf.reverse(x, axis = [2])
            flip2 = tf.reverse(tf.reverse(x, axis = [1]), axis = [2])
            flip3 = x
            pred0 = self.model(flip0, training = training)
            pred1 = self.model(flip1, training = training)
            pred2 = self.model(flip2, training = training)
            pred3 = self.model(flip3, training = training)

            pred0 = tf.reverse(pred0, axis = [1])
            pred1 = tf.reverse(pred1, axis = [2])
            pred2 = tf.reverse(tf.reverse(pred2, axis = [1]), axis = [2])
            pred = (pred0 + pred1 + pred2 + pred3) / 4
            return self.layer(pred)
        else:
            pred = self.model(x, training = training)
            return self.layer(pred)
    def __call__(self, x, training = True):
        return self.call(x, training = training)
def get_model(model_path):
    with STRATEGY.scope():
        if 'b2' in model_path or 'tpu' in model_path:
            model ='efficientnetb2'
        elif 'b4' in model_path:
            model = 'efficientnetb4'
        else:
            model = 'efficientnetb0'
        if 'effnet' in model_path or 'unet' in model_path or 'tpu' in model_path:
            model = Model('unet', model)
        elif 'fpn' in model_path:
            model = Model("fpn", model)
        else:
            model = Model("linknet", model)
        model.compile()
        model.build((1, width, height, 3))
        model.load_weights(model_path)
    return model

In [6]:
models = []
for model_idx in range(len(MODELS)):
    models += [get_model(MODELS[model_idx])]

## Mask to Rle and Rle to Mask

In [7]:
#functions to convert encoding to mask and mask to encoding
def enc2mask(encs, shape):
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for m,enc in enumerate(encs):
        if isinstance(enc,np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(height)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1 + m
    return img.reshape(shape).T

def mask2enc(mask, n=1):
    pixels = mask.T.flatten()
    encs = []
    for i in range(1,n+1):
        p = (pixels == i).astype(np.int8)
        if p.sum() == 0: encs.append(np.nan)
        else:
            p = np.concatenate([[0], p, [0]])
            runs = np.where(p[1:] != p[:-1])[0] + 1
            runs[1::2] -= runs[::2]
            encs.append(' '.join(str(x) for x in runs))
    return encs

#https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
#with transposed mask
def rle_encode_less_memory(img):
    #the image should be transposed
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

In [8]:
# Imagenet statistics Mean and variance
mean = np.array([0.63701495, 0.4709702, 0.6817423])
mean = np.expand_dims(np.expand_dims(mean, axis = 0), axis = 0)
std = np.array([0.15978882, 0.2245109, 0.14173926])
std = np.expand_dims(np.expand_dims(std, axis = 0), axis = 0)


identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
def normalize(image):
    image = image - mean
    image = image / std
    return image
def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4)

class HuBMAPDataset(tf.keras.utils.Sequence):
    def __init__(self, data):
        self.data = data
        if self.data.count != 3:
            subdatasets = self.data.subdatasets
            self.layers = []
            if len(subdatasets) > 0:
                for i, subdataset in enumerate(subdatasets, 0):
                    self.layers.append(rasterio.open(subdataset))
        self.shape = self.data.shape
        self.mask_grid = make_grid(self.data.shape, window=window, min_overlap=min_overlap)
        self.idx = 0
        self.actual_length = len(self.mask_grid)
    def __len__(self):
        if self.actual_length % batch_size == 0:
            return self.actual_length // batch_size
        else:
            return self.actual_length // batch_size + 1
        
    def __call__(self):
        if self.idx >= self.actual_length:
            return None, None
        x1, x2, y1, y2 = self.mask_grid[self.idx]
        if self.data.count == 3:
            img = np.zeros((3, window, window), dtype=np.uint8)
            dist_x = x2- x1
            dist_y = y2-y1
            img[:, :dist_x, :dist_y] = data.read([1,2,3], window=Window.from_slices((x1, x2), (y1, y2)))
            img = np.moveaxis(img, 0, -1)
        else:
            img = np.zeros((window, window, 3), dtype=np.uint8)
            dist_x = x2-x1
            dist_y = y2-y1
            for i, layer in enumerate(self.layers):
                img[:dist_x,:dist_y,i] = layer.read(window=Window.from_slices((x1,x2),(y1,y2)))
        img = cv2.resize(img, (height, width), cv2.INTER_AREA)
        img = img.astype(np.float32) / 255.0
        img = np.expand_dims(normalize(img).astype(np.float32), axis = 0)
        vetices = np.expand_dims(np.array([x1, x2, y1, y2]).astype(np.int32), axis = 0)
        self.idx += 1
        return (img.astype(np.float16), vetices)
    def __getitem__(self, idx):
        all_images = []
        all_labels = []
        for i in range(batch_size):
            image, labels = self()
            if image is None:
                break
            all_images += [image]
            all_labels += [labels]
        return np.concatenate(all_images, axis = 0), np.concatenate(all_labels, axis = 0)


Inference

In [9]:
import matplotlib.pyplot as plt
names, predictions = [],[]
for idx, row in tqdm(df_sample.iterrows(),total=len(df_sample)):
    imageId = row['id']
    data = rasterio.open(os.path.join(DATA, imageId+'.tiff'), transform = identity, num_threads='all_cpus')
    preds = np.zeros(data.shape, dtype=np.uint8)
    total_images = 0
    
    dataset = HuBMAPDataset(data)
    for i, (img, vertices) in enumerate(dataset):
        count = 0
        pred = tf.zeros((img.shape[0], img.shape[1], img.shape[2], P['NUM_CLASSES']))
        for model in models:
            p = model.predict(img) # Eval Mode
            pred = pred + p
            count += 1
        pred = pred / count
        pred = tf.keras.activations.softmax(pred, axis = -1)[:, :, :, 1]
        pred = pred.numpy().astype(np.float32)

        pred[:, :BORDER_ZEROS, :] = 0
        pred[:, :, :BORDER_ZEROS] = 0
        pred[:, -BORDER_ZEROS:, :] = 0
        pred[:, :, -BORDER_ZEROS:] = 0
        #for idx in range(len(img)):
        #    plt.imshow(img[idx].astype(np.float32))
        #    plt.imshow(pred[idx] > THRESHOLD, alpha = 0.5)
        #    plt.show()
        assert not np.isnan(np.sum(pred))
        for p, vert in zip(pred, vertices):
            x1, x2, y1, y2 = vert

            p = cv2.resize(p, (window, window))
            p = (p > THRESHOLD).astype(np.uint8)
            if np.sum(p) < MIN_PRED:
                p = np.zeros_like(p)
            diff_x = x2 - x1
            diff_y = y2 - y1
            preds[x1:x2,y1:y2] += p[:diff_x, :diff_y]

        total_images += len(img)
        print(total_images)

    del dataset
        
    preds = (preds >= VOTERS).astype(np.uint8)
    #convert to rle
    rle = rle_encode_less_memory(preds)
    
    names.append(imageId)
    predictions.append(rle)
    del preds
    gc.collect()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/5 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [10]:
df = pd.DataFrame({'id':names,'predicted':predictions}).set_index('id')
df = df.reset_index()
df.to_csv('submission.csv', index = False)